<a href="https://colab.research.google.com/github/trilgar/IAD2023/blob/main/%D0%97%D0%B0%D1%80%D0%B8%D1%86%D1%8C%D0%BA%D0%B8%D0%B9_CP11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><font size="6"><b>Комп'ютерний практикум 11.

<center><b> Колаборативна фільтрація </font>


<center><b><i><font size="4"> Collaborative Filtering



</b></center>



##<center>__Самостійні завдання__</center>

> Скопіювати блок самостійних завдань в окремий файл ***LastName_CP11.ipynb***

> Інсталюйте необхідні пакети бібліотек Python

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%matplotlib inline

### Завдання №1

* Завантажте архів `BX-CSV-Dump.zip` та розпакуйте дані з лінку

`url = "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"`

* Завантажте датасети `BX-Books.csv` як `df.books` та `BX-Book-Ratings.csv` як `df.ratings`, виведіть частину датасетів на екран

Використайте ці дані, щоб побудувати систему рекомендацій, яка підкаже, що користувачеві варто прочитати наступним, виходячи з його поточних книжкових уподобань.


 


In [2]:
import requests
import zipfile
import io

url = "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"
response = requests.get(url)

zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall()

In [3]:
# Завантаження датасету BX-Books.csv
books_file = "BX-Books.csv"
df_books = pd.read_csv(books_file, sep=";", on_bad_lines="skip", encoding="latin-1")
print(df_books.head())

# Завантаження датасету BX-Book-Ratings.csv
ratings_file = "BX-Book-Ratings.csv"
df_ratings = pd.read_csv(ratings_file, sep=";", on_bad_lines="skip", encoding="latin-1")
print(df_ratings.head())

/tmp/ipykernel_458355/3652178068.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv(books_file, sep=";", on_bad_lines="skip", encoding="latin-1")


         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

### Завдання №2

* Виведіть інформацію про датасет
* Залиште в датасеті лише колонки `ISBN`, `Book Title`,`Book Author` and `Publisher`.
* Перевірте, чи немає дублікатів назв книг, видаліть, якщо такі є. Для цього скористайтесь функціями `duplicated(subset='Book-Title')`  та `drop_duplicates(subset='Book-Title')`
* Об'єднайте імена та прізвища авторів, видаляючи пробіли, щоб система розрізняла авторів з однаковими іменами
* Виведіть шапку таблиці, щоб переконатись у об'єднанні



In [4]:
# Вибираємо потрібні колонки
selected_columns = ['ISBN', 'Book-Title', 'Book-Author', 'Publisher']
df_books_selected = df_books[selected_columns]

# Видаляємо дублікати назв книг
df_books_selected = df_books_selected.drop_duplicates(subset='Book-Title')

# Об'єднуємо імена та прізвища авторів, видаляючи пробіли
df_books_selected['Book-Author'] = df_books_selected['Book-Author'].str.replace(' ', '')

In [5]:
df_books_selected.head()

,ISBN,Book-Title,Book-Author,Publisher
0,0195153448,Classical Mythology,MarkP.O.Morford,Oxford University Press
1,0002005018,Clara Callan,RichardBruceWright,HarperFlamingo Canada
2,0060973129,Decision in Normandy,CarloD'Este,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,GinaBariKolata,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E.J.W.Barber,W. W. Norton &amp; Company


In [6]:
df_ratings = df_ratings.rename(columns={'Book-Rating': 'Rating'})
df_ratings.head()

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Завдання №3

* Створіть користувача, якому поставте у відповідність 5 довільних кних з переліку та рейтингом (0-10)
* Створіть рекомендований список з 10 книг для користувача, спираючись на автора та видавництво.


In [7]:
user_preferences = {
    'ISBN': ['0004722124', '8511839102', '052165615X', '0521795028', '0515107662'],
    'Rating': [8, 7, 6, 9, 5]
}

inputBooks = pd.DataFrame(user_preferences)
inputBooks

,ISBN,Rating
0,0004722124,8
1,8511839102,7
2,052165615X,6
3,0521795028,9
4,0515107662,5


In [8]:
userSubset = df_ratings[df_ratings['ISBN'].isin(inputBooks['ISBN'].tolist())]
userSubsetGroup = userSubset.groupby(['User-ID'])
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)
userSubsetGroup = userSubsetGroup[0:100]

In [9]:
#Зберігаємо кореляцію Пірсона у словнику, де ключем є ідентифікатор користувача, а значенням - коефіцієнт
pearsonCorrelationDict = {}

#Для кожної групи користувачів у нашій підмножині
for name, group in userSubsetGroup:
    name = name[0]
    #Почнемо з сортування вхідних даних та поточної групи користувачів, щоб не переплутати значення
    group = group.sort_values(by='ISBN')
    inputBooks = inputBooks.sort_values(by='ISBN')
    #Отримаємо N для формули
    nRatings = len(group)
    #Отримаємо оцінки на фільми, які мають спільні риси
    temp_df = inputBooks[inputBooks['ISBN'].isin(group['ISBN'].tolist())]
    #Збережемо їх у тимчасовій буферній змінній у форматі списку, щоб полегшити майбутні обчислення
    tempRatingList = temp_df['Rating'].tolist()
    #Виведемо оцінки поточної групи користувачів у вигляді списку
    tempGroupList = group['Rating'].tolist()
    #Обчислимо кореляцію Пірсона між двома користувачами, так званими, x та y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #Якщо знаменник відмінний від нуля, то ділимо, інакше кореляція 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [10]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User-ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,User-ID
0,1.0,276729
1,0.0,1733
2,0.0,14422
3,0.0,16795
4,0.0,26544


In [11]:
topUsers = pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,User-ID
0,1.0,276729
1,0.0,1733
26,0.0,157116
27,0.0,160878
28,0.0,170845


In [12]:
topUsersRating = topUsers.merge(df_ratings, left_on='User-ID', right_on='User-ID', how='inner')
topUsersRating.head()

,similarityIndex,User-ID,ISBN,Rating
0,1.0,276729,052165615X,3
1,1.0,276729,0521795028,6
2,0.0,1733,0060536705,7
3,0.0,1733,0061042005,9
4,0.0,1733,0061082945,0


In [13]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Rating']
topUsersRating.head()

,similarityIndex,User-ID,ISBN,Rating,weightedRating
0,1.0,276729,052165615X,3,3.0
1,1.0,276729,0521795028,6,6.0
2,0.0,1733,0060536705,7,0.0
3,0.0,1733,0061042005,9,0.0
4,0.0,1733,0061082945,0,0.0


In [14]:
#Знайдемо суму до topUsers після групування за userId
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
ISBN,,
000123207X,0.0,0.0
0001232088,0.0,0.0
0001931377,0.0,0.0
0001981625,0.0,0.0
0002220296,0.0,0.0


In [15]:
#Створюємо порожній фрейм даних
recommendation_df = pd.DataFrame()
#Тепер беремо середньозважене значення
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,ISBN
ISBN,,
000123207X,NaN,000123207X
0001232088,NaN,0001232088
0001931377,NaN,0001931377
0001981625,NaN,0001981625
0002220296,NaN,0002220296


In [16]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,ISBN
ISBN,,
0521795028,6.0,0521795028
052165615X,3.0,052165615X
000123207X,NaN,000123207X
0001232088,NaN,0001232088
0001931377,NaN,0001931377
0001981625,NaN,0001981625
0002220296,NaN,0002220296
0002241447,NaN,0002241447
000225218X,NaN,000225218X


In [17]:
#Підсумкова таблиця рекомендацій
df_books_selected.loc[df_books_selected['ISBN'].isin(recommendation_df.head(10)['ISBN'].tolist())]

,ISBN,Book-Title,Book-Author,Publisher
134860,000123207X,Paddington's Birthday Party,MichaelBond,HarperCollins Publishers
134864,0001232088,Paddington in the Kitchen,MichaelBond,HarperCollins Publishers
136388,000225218X,'Tis the Season to Be Crabby (Festive Peanuts),CharlesM.Schulz,HarperEntertainment
160137,0001981625,The Sea of Tranquility,MarkHaddon,HarperCollins Publishers
246838,052165615X,Help!: Level 1,PhilipProwse,Cambridge University Press
246839,0521795028,The Amsterdam Connection : Level 4 (Cambridge ...,SueLeather,Cambridge University Press
